# LLM aids for processing of the first Carlson-Putin interview 

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
[RakuForPrediction-book at GitHub](https://github.com/antononcube/RakuForPrediction-book)   
February 2024

-------

## Introduction

In this notebook we provide aids and computational workflows for the analysis of the first Carlson-Putin interview held in February 9th, 2024.
We mostly use Large Language Models (LLMs). We walk through various steps involved in examining and understanding the interview in a systematic and reproducible manner.

The interview transcripts are taken from [en.kremlin.ru](http://en.kremlin.ru).

The computations are done with a [Raku chatbook](https://raku.land/zef:antononcube/Jupyter::Chatbook), [AAp6, AAv1÷AAv3]. The LLM functions used in the workflows are explained and demonstrated in [AA1, AAv3].
The workflows are done with OpenAI's models [AAp1]; the models of Google (PaLM), [AAp2], and MistralAI, [AAp3], can be also used for the Part 1 summary and the search engine.
The related images were generates with workflows described in [AA2].

### Structure

The structure of the notebook is as follows:

1. **Getting the interview text**   
    Standard ingestion.
2. **Preliminary LLM queries**   
    What are the most important parts or most provocative questions?
3. **Part 1: separation and summary**   
    Overview of the historical review.
4. **Part 2: thematic parts**   
    TLDR via a table of themes.
5. **Interview's spoken parts**    
    Non-LLM extraction of participants' parts.
6. **Search engine**   
    Fast results with LLM embeddings.
7. **Flavored variations**   
    How would Hillary phrase it? And how would Trump answer it?

Sections 5 and 6 can be skipped -- they are (somewhat) more technical.

### Observations

- Using the LLM functions for programmatic access of LLMs speeds up the efforts, I would say, by factor 3-5 times.
- The workflows presented below are fairly universal -- with small changes the notebook can be applied to other interviews.
- Using OpenAI's preview model "gpt-4-turbo-preview" spares or simplifies a fair amount of workflow elements.
    - The model "gpt-4-turbo-preview" takes input with 128K tokens, [OAIb1]. 
    - Hence the whole interview can be processed in one LLM request.
- Since I watched the interview, I can see that the LLM results for most provocative questions or most important statements are good.
    - It is interesting to think about believing those results by people who have not watched the interview.
- The search engine can be replaced or enhanced with a Question Answering System (QAS).
- The flavored variations might be too subtle.
    - I expected more obvious manifestation of characters involved.

-------

## Getting the interview text

The interviews are taken from the dedicated Kremlin's page ["Interview to Tucker Carlson"](http://en.kremlin.ru/events/president/news/73411), hosted at [en.kremlin.ru](http://en.kremlin.ru).

Here we load a packages and define a text statistics function and a display function:

In [1]:
from LLMFunctionObjects import *
from LLMPrompts import *
from DataTypeSystem import *
import math
import json
import pandas as pd
import random
from IPython.display import display, Markdown, Latex


In [2]:
def text_stats(txt: str) -> dict:
    return {"chars": len(txt), "words": len(txt.split()), "lines": len(txt.splitlines())}

def multi_column(data, cols=2):
    rows = math.ceil(len(data) / cols)
    return pd.DataFrame([data[i:i + rows] for i in range(0, len(data), rows)]).transpose()

def from_json(data):
    res = data.replace("```json","").replace("```","").strip()
    return json.loads(res)

Here we set display options for data frames:

In [3]:
pd.set_option('display.max_colwidth', None)

Here we ingest interview's text:

In [4]:
import requests
import re

def text_stats(txt: str) -> dict:
    return {"chars": len(txt), "words": len(txt.split()), "lines": len(txt.splitlines())}

url = 'https://raw.githubusercontent.com/antononcube/SimplifiedMachineLearningWorkflows-book/master/Data/Carlson-Putin-interview-2024-02-09-English.txt'
response = requests.get(url)
txtEN = response.text
txtEN = re.sub(r'\n+', "\n", txtEN)

print(text_stats(txtEN))

{'chars': 97354, 'words': 16980, 'lines': 292}


------

## Preliminary LLM queries

Here we configure LLM access -- we use OpenAI's model "gpt-4-turbo-preview" since it allows inputs with 128K tokens:

In [5]:
conf = llm_configuration('ChatGPT', model = 'gpt-4-turbo-preview', max_tokens = 4096, temperature = 0.3)
len(conf.to_dict())

conf4 = llm_configuration('ChatGPT', model = 'gpt-4-1106-preview', max_tokens = 4096, temperature = 0.3)
len(conf4.to_dict())

23

### Questions

First we make an LLM request about the number of questions asked:

In [6]:
llm_synthesize(["How many questions were asked in the following interview?\n\n", txtEN], e = conf)

'In the interview between Tucker Carlson and President Vladimir Putin, Tucker Carlson asked a total of 52 questions.'

Here we ask the questions to be extracted into a JSON list:

In [7]:
llmQuestions = llm_synthesize([
    "Extract all questions from the following interview into a JSON list.\n\n", 
    txtEN,
    llm_prompt('NothingElse')('JSON')
    ], e = conf, form = sub_parser('JSON', drop=True))

deduce_type(llmQuestions)

Atom(<class 'str'>)

In [9]:
sp = sub_parser('JSON', drop=True)
print(sp.process(llmQuestions))

[{'question': 'Tell us why you believe the United States might strike Russia out of the blue. How did you conclude that?'}, {'question': 'You were initially trained in history, as far as I know?'}, {'question': 'I beg your pardon, can you tell us what period… I am losing track of where in history we are.'}, {'question': 'May I ask… You are making the case that Ukraine, certain parts of Ukraine, Eastern Ukraine, in fact, has been Russia for hundreds of years. Why wouldn’t you just take it when you became President 24 years ago? You have nuclear weapons, they don’t. It’s actually your land. Why did you wait so long?'}, {'question': 'You have, I see, encyclopaedic knowledge of that region. But why didn’t you make this case for the first 22 years as president, that Ukraine wasn’t a real country?'}, {'question': 'Do you believe Hungary has a right to take back its land from Ukraine? And that other nations have a right to go back to their 1654 borders?'}, {'question': 'Have you told Viktor O

In [8]:
multi_column(from_json(llmQuestions), 3)

,0,1,2
0,Tell us why you believe the United States might strike Russia out of the blue. How did you conclude that?,Have you told Viktor Orban that he can have a part of Ukraine?,Do you have evidence that NATO or the CIA did it?
1,"You were initially trained in history, as far as I know?",May I ask what year was this?,What do you think of that?
2,"I beg your pardon, can you tell us what period… I am losing track of where in history we are.","So, twice you've described US presidents making decisions and then being undercut by their agency heads. So, it sounds like you're describing a system that is not run by the people who are elected, in your telling.","Evan Gershkovich who is the Wall Street Journal reporter, he is 32 and he’s been in prison for almost a year. This is a huge story in the United States and I just want to ask you directly without getting into details of your version of what happened, if as a sign of your decency you’ll be willing to release him to us and we’ll bring him back to the United States?"
3,"It doesn’t sound like you are inventing it, but I am not sure why it’s relevant to what’s happened two years ago.","Well, of course, it did come true, and you’ve mentioned it many times. I think, it’s a fair point. And many in America thought that relations between Russia and the United States would be fine after the collapse of the Soviet Union, at the core. But the opposite happened. But have never explained why you think that happened, except to say that the West fears a strong Russia. But we have a strong China that the West doesn’t seem to be very afraid of. What about Russia, what do you think convinced the policymakers to take it down?","Are you suggesting he was working for the US government or NATO? Or he was just a reporter who was given material he wasn’t supposed to have? Those seem like very different, very different things."
4,"May I ask… You are making the case that Ukraine, certain parts of Ukraine, Eastern Ukraine, in fact, has been Russia for hundreds of years. Why wouldn’t you just take it when you became President 24 years ago? You have nuclear weapons, they don’t. It’s actually your land. Why did you wait so long?",When was the last time you spoke to Joe Biden?,Do you think it is too humiliating at this point for NATO to accept Russian control of what was two years ago Ukrainian territory?
5,Were you sincere? Would you have joined NATO?,You do not remember?!,"Would you be willing to say, “Congratulations, NATO, you won?” And just keep the situation where it is now?"
6,"But if he had said yes, would you have joined NATO?","But he is funding the war that you are fighting, so I think that would be memorable?",Do you think it is too humiliating at this point for NATO to accept Russian control of what was two years ago Ukrainian territory?
7,"Why do you think that is? Just to get to motive. I know, you’re clearly bitter about it. I understand. But why do you think the West rebuffed you then? Why the hostility? Why did the end of the Cold War not fix the relationship? What motivates this from your point of view?",Do you think NATO was worried about this becoming a global war or nuclear conflict?,"Would you be willing to say, “Congratulations, NATO, you won?” And just keep the situation where it is now?"
8,Do you believe Hungary has a right to take back its land from Ukraine? And that other nations have a right to go back to their 1654 borders?,Who blew up Nord Stream?,None


We can see that the LLM-extracted questions are two times less than the LLM-obtained number of questions above.
Here are the extracted questions (in a multi-column format): 

### Important parts

Here we make a function of extracting *significant* parts from the interview:

In [9]:
# fProv = llm_function(lambda x, y, z: f"Put the top {x} most {y} from the following interview in a Markdown list. {z}", e = conf)
fProv = llm_function(lambda x, y, z: f"Put the top {x} most {y} from the following interview in a JSON list. {z}", e = conf)

#### Most provocative questions

Here we get the most provocative questions:

In [10]:
llmTopQuestions = fProv(3, "provocative questions", txtEN)

In [11]:
pd.DataFrame(from_json(llmTopQuestions))

,question,answer
0,What is denazification?,Denazification means the prohibition of all kinds of neo-Nazi movements.
1,Do you think Zelensky has the freedom to negotiate the settlement to this conflict?,"I don’t know the details, of course it’s difficult for me to judge, but I believe he has, in any case, he used to have."
2,"Would you be willing to say, “Congratulations, NATO, you won?” And just keep the situation where it is now?","You know, it is a subject for the negotiations no one is willing to conduct or, to put it more accurately, they are willing but do not know how to do it."


#### Most important statements

Here we get the most important statements:

In [12]:
llmTopStatements = fProv(3, 'important statements', txtEN)

In [13]:
pd.DataFrame(from_json(llmTopStatements))

,statement,context
0,"We are willing to negotiate. It is the Western side, and Ukraine is obviously a satellite state of the US. It is evident. I do not want you to take it as if I am looking for a strong word or an insult, but we both understand what is happening.","Putin expresses readiness for negotiations and characterizes Ukraine as a satellite state of the US, indicating a willingness to engage in dialogue to resolve the conflict."
1,"I do not rule out that the person you referred to, Mr. Gershkovich, may return to his homeland. I am absolutely sincere. But let me say once again, the dialogue continues. The more public we render things of this nature, the more difficult it becomes to resolve them. Everything has to be done in a calm manner.","Putin discusses the possibility of releasing Evan Gershkovich, a Wall Street Journal reporter imprisoned in Russia, highlighting ongoing dialogue and the need for calm negotiations."
2,"It is very easy: when it comes to protecting oneself and one’s family, one’s homeland. We don’t attack anyone. When did the developments in Ukraine start? Since the coup d’état and the hostilities in Donbass began, that’s when they started. And we are protecting our people, ourselves, our homeland and our future.","Putin justifies military actions in Ukraine as a form of self-defense following the 2014 coup d’état, emphasizing the protection of Russian people and interests."


-------

## Part 1: separation and summary

In the first part of the interview Putin gave a historical overview of the formation and evolution of the "Ukrainian lands."
We can extract the first part of the interview "manually" like this:

In [14]:
part1, part2 = txtEN.split('Tucker Carlson: Do you believe Hungary has a right to take back its land from Ukraine?')

print(f"Part 1 stats: {text_stats(part1)}")
print(f"Part 2 stats: {text_stats(part2)}")

Part 1 stats: {'chars': 13629, 'words': 2300, 'lines': 44}
Part 2 stats: {'chars': 83639, 'words': 14664, 'lines': 248}


Alternatively, we can ask ChatGPT to make the extraction for us:

In [15]:
splittingQuestion = llm_synthesize([
    "Which question by Tucker Carlson splits the following interview into two parts:",
    "(1) historical overview Ukraine's formation, and (2) shorter answers.", 
    txtEN,
    llm_prompt('NothingElse')('the splitting question by Tucker Carlson')
    ], e = conf)

In [16]:
splittingQuestion

'Tucker Carlson: May I ask… You are making the case that Ukraine, certain parts of Ukraine, Eastern Ukraine, in fact, has been Russia for hundreds of years. Why wouldn’t you just take it when you became President 24 years ago? You have nuclear weapons, they don’t. It’s actually your land. Why did you wait so long?'

Here is the first part of the interview according to the LLM result:

In [17]:
splittingQuestion[10:200]

'lson: May I ask… You are making the case that Ukraine, certain parts of Ukraine, Eastern Ukraine, in fact, has been Russia for hundreds of years. Why wouldn’t you just take it when you becam'

In [18]:
llmPart1 = txtEN.split(splittingQuestion[10:200])[0]
text_stats(llmPart1)

{'chars': 8736, 'words': 1483, 'lines': 29}

**Remark:** We can see that LLM "spared" nearly 1/3 of the "manually" selected text. Below we continue with the latter.

### Summary of the first part

Here is a summary of the first part of the interview:

In [19]:
res = llm_synthesize(["Summarize the following part one of the Carlson-Putin interview:", part1], e = conf)

In [20]:
display(Markdown(res))

In the first part of the interview between Tucker Carlson and President Vladimir Putin, Carlson questions Putin about his statement regarding the United States and NATO potentially launching a surprise attack on Russia, which Putin clarifies as a misunderstanding. Putin then delves into a detailed historical account to explain the origins and development of the relationship between Russia and Ukraine. He traces back to the establishment of the Russian state in 862 with the invitation of Rurik to reign in Novgorod, the significance of Kiev in Russian history, the baptism of Russia in 988, and the eventual fragmentation and reunification of Russian lands over centuries.

Putin discusses the influence of the Grand Duchy of Lithuania and the Polish-Lithuanian Commonwealth on Ukrainian territories, the efforts of Polonization, and the resistance and appeals of local leaders to Moscow for protection and integration into the Russian state. He highlights key historical events, including the Treaty of Pereyaslav in 1654, which brought parts of Ukraine under Russian rule, and the subsequent wars and treaties that shaped the borders of Russia and Poland.

Putin also touches on the role of the Austrian General Staff and Ukrainian nationalists in promoting Ukrainian identity separate from Russian identity, leading up to the Bolsheviks' establishment of Soviet Ukraine and the territorial adjustments made by Lenin and Stalin, which included lands that historically were not part of Ukraine. He argues that these historical contexts demonstrate that Ukraine is an "artificial state" shaped by various historical forces and decisions.

Throughout the discussion, Carlson interjects with questions to clarify the timeline and relevance of Putin's historical account to the current conflict in Ukraine, expressing interest in Putin's detailed knowledge but questioning why these arguments about Ukraine's legitimacy as a country were not prominently made earlier in Putin's presidency.

-----

## Part 2: thematic parts

Here we make an LLM request for finding and distilling the themes or the second part of the interview:

In [21]:
llmParts = llm_synthesize([
    'Split the following second part of the Tucker-Putin interview into thematic parts:', 
    part2,
    "Return the parts as a JSON array.",
    llm_prompt('NothingElse')('JSON')
    ], e = conf)

In [22]:
llmParts2 = from_json(llmParts)
deduce_type(llmParts2)

Vector(Assoc(Atom(<class 'str'>), Atom(<class 'str'>), 2), 11)

Here we tabulate the found themes:

In [23]:
pd.DataFrame(llmParts2)

,theme,content
0,Historical Context and Borders,Vladimir Putin discusses the historical context of Russia's borders and the possibility of nations reclaiming lands based on historical precedents.
1,Dialogue with Hungary and Historical Anecdote,Putin denies discussing territorial claims with Viktor Orban and shares a personal story from the 1980s highlighting the presence of Hungarians in Ukraine.
2,Russia's Relations and NATO Expansion,"The discussion covers Russia's perspective on its relations with Ukraine and other countries post-Soviet Union collapse, emphasizing the issue of NATO expansion."
3,Perceived Threats and Security Concerns,"Putin expresses concerns about the security threats posed by NATO and the West, including the potential for nuclear threats."
4,Economic and Cultural Ties,"The conversation touches on the deep economic and cultural ties between Russia and Ukraine, suggesting these as a basis for inevitable good relations."
5,Western Relations and Missed Opportunities,"Putin reflects on missed opportunities for cooperation with the West and the expansion of NATO, criticizing Western actions."
6,Russia's Military Actions in Ukraine,"Putin discusses the reasons behind Russia's military actions in Ukraine, framing it as a response to protect Russian speakers and counteract NATO's influence."
7,Global Power Dynamics,"The interview explores the shift in global power dynamics, with Putin commenting on the rise of China and other countries, and the implications for the US."
8,Artificial Intelligence and Genetic Engineering,Putin and Carlson discuss the potential threats and ethical considerations of advancements in artificial intelligence and genetic engineering.
9,Religious and Cultural Identity,Putin speaks on the importance of Orthodox Christianity in Russian identity and its role in shaping the country's values and outlook.


------

## Interview's spoken parts

In this section we separate the spoken parts of each participant in the interview. We do that using regular expressions, not LLMs.

Here we split the interview text with the names of the participants:

In [24]:
parts = list(filter(None, re.split(r"(Tucker Carlson:)|(President of Russia Vladimir Putin:)|(Vladimir Putin:)", txtEN)))
parts = list(zip(parts[::2], parts[1::2]))

print("Total parts", len(parts))

print("First 4:")
for part in parts[:4]:
    print(part)

Total parts 149
First 4:
('Tucker Carlson:', ' Mr. President, thank you.\nOn February 24, 2022, you addressed your country in your nationwide address when the conflict in Ukraine started and you said that you were acting because you had come to the conclusion that the United States through NATO might initiate a quote, “surprise attack on our country.” And to American ears that sounds paranoid. Tell us why you believe the United States might strike Russia out of the blue. How did you conclude that?\n')
('President of Russia Vladimir Putin:', " It's not that the United States was going to launch a surprise strike on Russia, I didn't say so. Are we having a talk show or a serious conversation?\n")
('Tucker Carlson:', ' That was a good quote. Thank you, it’s formidably serious!\n')
('Vladimir Putin:', ' You were initially trained in history, as far as I know?\n')


Here we further process the separate participant names and corresponding parts into a list of pairs:

In [25]:
parts2 = [(('T.Carlson' if 'Carlson' in part[0] else 'V.Putin'), part[1].strip()) for part in parts]
pd.DataFrame(parts2)

,0,1
0,T.Carlson,"Mr. President, thank you.\nOn February 24, 2022, you addressed your country in your nationwide address when the conflict in Ukraine started and you said that you were acting because you had come to the conclusion that the United States through NATO might initiate a quote, “surprise attack on our country.” And to American ears that sounds paranoid. Tell us why you believe the United States might strike Russia out of the blue. How did you conclude that?"
1,V.Putin,"It's not that the United States was going to launch a surprise strike on Russia, I didn't say so. Are we having a talk show or a serious conversation?"
2,T.Carlson,"That was a good quote. Thank you, it’s formidably serious!"
3,V.Putin,"You were initially trained in history, as far as I know?"
4,T.Carlson,Yes.
...,...,...
144,T.Carlson,Do you think it is too humiliating at this point for NATO to accept Russian control of what was two years ago Ukrainian territory?
145,V.Putin,"I said let them think how to do it with dignity. There are options if there is a will.\nUp until now there has been the uproar and screaming about inflicting a strategic defeat on Russia on the battlefield. Now they are apparently coming to realize that it is difficult to achieve, if possible at all. In my opinion, it is impossible by definition, it is never going to happen. It seems to me that now those who are in power in the West have come to realize this as well. If so, if the realization has set in, they have to think what to do next. We are ready for this dialogue."
146,T.Carlson,"Would you be willing to say, “Congratulations, NATO, you won?” And just keep the situation where it is now?"
147,V.Putin,"You know, it is a subject for the negotiations no one is willing to conduct or, to put it more accurately, they are willing but do not know how to do it. I know they want. It is not just that I see it but I know they do want it but they are struggling to understand how to do it. They have driven the situation to the point where we are at. It is not us who have done that, it is our partners, opponents who have done that. Well, now let them think how to reverse the situation. We are not against it.\nIt would be funny if it were not so sad. This endless mobilization in Ukraine, the hysteria, the domestic problems – sooner or later it all will result in an agreement. You know, this will probably sound strange given the current situation but the relations between the two peoples will be rebuilt anyway. It will take a lot of time but they will heal.\nI will give you very unusual examples. There is a combat encounter on the battlefield, it is a specific example: Ukrainian soldiers got encircled (this is an example from real life), our soldiers were shouting to them: “There is no chance! Surrender yourselves! Come out and you will be alive!” Suddenly the Ukrainian soldiers were shouting back in Russian, perfect Russian: “Russians never surrender!” and all of them perished. They still identify themselves as Russians.\nWhat is happening is, to a certain extent, an element of a civil war. Everyone in the West thinks that the Russian people have been split by hostilities forever. No. They will be reunited. The unity is still there.\nWhy are the Ukrainian authorities dismantling the Ukrainian Orthodox Church? Because it unites not the territory, it unites our souls. No one will be able to disunite them.\nShall we end here or there is something else?"


Here we get all spoken parts of Tucker Carlson (and consider all of them to be "questions"):

In [26]:
tcQuestions = [part[1] for part in parts2 if 'Carlson' in part[0] and part[1].endswith('?')]
len(tcQuestions)

53

A table with all questions:

In [27]:
multi_column(tcQuestions, 3)

,0,1,2
0,"Mr. President, thank you.\nOn February 24, 2022, you addressed your country in your nationwide address when the conflict in Ukraine started and you said that you were acting because you had come to the conclusion that the United States through NATO might initiate a quote, “surprise attack on our country.” And to American ears that sounds paranoid. Tell us why you believe the United States might strike Russia out of the blue. How did you conclude that?",What is denazification? What would that mean?,"Well, let’s just give one example — the US dollar, which has, kind of, united the world in a lot of ways, maybe not to your advantage, but certainly to ours. Is that going away as the reserve currency, the universally accepted currency? How have sanctions, do you think, changed the dollar’s place in the world?"
1,"May I ask… You are making the case that Ukraine, certain parts of Ukraine, Eastern Ukraine, in fact, has been Russia for hundreds of years. Why wouldn’t you just take it when you became President 24 years ago? You have nuclear weapons, they don’t. It’s actually your land. Why did you wait so long?",Would you be satisfied with the territory that you have now?,"I think that is a fair assessment. The question is what comes next? And maybe you trade one colonial power for another, much less sentimental and forgiving colonial power? Is the BRICS, for example, in danger of being completely dominated by the Chinese economy? In a way that is not good for their sovereignty. Do you worry about that?"
2,In 1654?,"Really, my question is: What do you do about it? I mean, Hitler has been dead for eighty years, Nazi Germany no longer exists, and it’s true. So, I think, what you are saying, you want to extinguish or at least control Ukrainian nationalism. But how do you do that?","So, you said a moment ago that the world would be a lot better if it were not broken into competing alliances, if there was cooperation globally. One of the reasons you don’t have that is because the current American administration is dead set against you. Do you think if there was a new administration after Joe Biden that you would be able to re-establish communication with the US government? Or does it not matter who the President is?"
3,"You have, I see, encyclopaedic knowledge of that region. But why didn’t you make this case for the first 22 years as president, that Ukraine wasn’t a real country?","Right. My question is almost specific, it was, of course, not a defense of Nazism. Otherwise, it was a practical question. You don't control the entire country, you don’t seem like you want to. So, how do you eliminate that culture, or an ideology, or feelings, or a view of history, in a country that you don’t control? What do you do about that?","But you are describing two different systems. You say that the leader acts in the interests of the voters, but you also say that these decisions are not made by the leader – they are made by the ruling classes. You have run this country for so long, you have known all these American presidents. What are those power centres in the United States, do you think? And who actually makes the decisions?"
4,Do you believe Hungary has a right to take back its land from Ukraine? And that other nations have a right to go back to their 1654 borders?,"Well, but you would not be speaking to the Ukrainian president, you would be speaking to the American president. When was the last time you spoke to Joe Biden?",I just have to ask. You have said clearly that NATO expansion eastward is a violation of the promise you were all made in the 1990s. It is a threat to your country. Right before you sent troops into Ukraine the Vice-President of the United States spoke at the Security Conference and encouraged the President of Ukraine to join NATO. Do you think that was an effort to provoke you into military action?
5,Have you told Viktor Orban that he can have a part of Ukraine?,"But he is funding the war that you are fighting, so I t

-------

## Search engine

In this section we make a (mini) search engines of the interview parts obtained above.

Here are the steps:

1. Make sure the interview parts are associated with unique identifiers that also identify the speakers
2. Find the embedding vectors for each part.
3. Create a recommendation function that:
   - Filters the embeddings according to specified type 
   - Finds the vector embedding of given query
   - Finds the dot products of query-vector with the part-vectors
   - Pick the top results

Here we make a hash-map of the interview parts obtained above:

In [28]:
k = 0
parts = {f"{k} {key}": value for k, (key, value) in enumerate(parts)}
len(parts)

149

Here we find the LLM embedding vectors of the interview parts:

In [29]:
from openai import OpenAI
client = OpenAI()

embs = {key: client.embeddings.create(input=value, model = "text-embedding-3-large").data[0].embedding for key, value in parts.items()}
len(embs)


149

Here is a function to find the most relevant parts of the interview for a given query (using dot product):

In [30]:
def top_parts(query, n=3, type='answers'):
    vec = client.embeddings.create(input=query, model = "text-embedding-3-large").data[0].embedding

    if type is None:
        type = 'part'

    if type in ['part', 'statement']:
        embsLocal = embs
    elif type in ['answer', 'answers', 'Putin']:
        embsLocal = {key: value for key, value in embs.items() if 'Putin' in key}
    elif type in ['question', 'questions', 'Carlson', 'Tucker']:
        embsLocal = {key: value for key, value in embs.items() if 'Carlson' in key}
    else:
        raise ValueError(f"Do not know how to process the {type} argument.")

    sres = {key: sum([v1*v2 for v1, v2 in zip(value, vec)]) for key, value in embsLocal.items()}

    sres = sorted(sres.items(), key=lambda x: -x[1])
    return [{'Score': score, 'Text': parts[key]} for key, score in sres[:n]]


Here we find the top 3 results for a query:

In [31]:
res1 = top_parts("Who blew up NordStream 1 and 2?", 3, type = 'part')
pd.DataFrame(res1)

,Score,Text
0,0.878272,Who blew up Nord Stream?\n
1,0.500830,I was busy that day. I did not blow up Nord Stream.\n
2,0.443272,"This also confuses me. But today's German leadership is guided by the interests of the collective West rather than its national interests, otherwise it is difficult to explain the logic of their action or inaction. After all, it is not only about Nord Stream-1, which was blown up, and Nord Stream-2 was damaged, but one pipe is safe and sound, and gas can be supplied to Europe through it, but Germany does not open it. We are ready, please.\nThere is another route through Poland, called Yamal-Europe, which also allows for a large flow. Poland has closed it, but Poland pecks from the German hand, it receives money from pan-European funds, and Germany is the main donor to these pan-European funds. Germany feeds Poland to a certain extent. And they closed the route to Germany. Why? I don't understand.\nUkraine, to which the Germans supply weapons and give money. Germany is the second sponsor after the United States in terms of financial aid to Ukraine. There are two gas routes through Ukraine. They simply closed one route, the Ukrainians. Open the second route and get gas from Russia. They do not open it. Why don't the Germans say: “Look, guys, we give you money and weapons. Open up the valve, please, let the gas from Russia pass through for us.\nWe are buying liquefied gas at exorbitant prices in Europe, which brings the level of our competitiveness, and economy in general down to zero. Do you want us to give you money? Let us have a decent existence, make money for our economy, because this is where the money we give you comes from.” They refuse to do so. Why? Ask them. (Knocks on the table.) That is what it is like in their heads. Those are highly incompetent people.\n"


In [32]:
res2 = top_parts('Where the Russia-Ukraine negotiations were held?', 2, type = 'answer') 
pd.DataFrame(res2)

,Score,Text
0,0.510021,"They have been. They reached a very high stage of coordination of positions in a complex process, but still they were almost finalized. But after we withdrew our troops from Kiev, as I have already said, the other side (Ukraine) threw away all these agreements and obeyed the instructions of Western countries, European countries, and the United States to fight Russia to the bitter end.\nMoreover, the President of Ukraine has legislated a ban on negotiating with Russia. He signed a decree forbidding everyone to negotiate with Russia. But how are we going to negotiate if he forbade himself and everyone to do this? We know that he is putting forward some ideas about this settlement. But in order to agree on something, we need to have a dialogue. Is not that right?\n"
1,0.447505,"Initially, it was the coup in Ukraine that provoked the conflict.\nBy the way, back then the representatives of three European countries – Germany, Poland and France – arrived. They were the guarantors of the signed agreement between the Government of Yanukovych and the opposition. They signed it as guarantors. Despite that, the opposition staged a coup and all these countries pretended that they didn’t remember that they were guarantors of a peaceful settlement. They just threw it in the stove right away and nobody recalls that.\nI don’t know if the US know anything about that agreement between the opposition and the authorities and its three guarantors who, instead of bringing this whole situation back in the political field, supported the coup. Although, it was meaningless, believe me. Because President Yanukovych agreed to all conditions, he was ready to hold early election which he had no chance to win, frankly speaking. Everyone knew that.\nBut then why the coup, why the victims? Why threaten Crimea? Why launch an operation in Donbass? This I do not understand. That is exactly what the miscalculation is. The CIA did its job to complete the coup. I think one of the Deputy Secretaries of State said that it cost a large sum of money, almost 5 billion dollars. But the political mistake was colossal! Why would they have to do that? All this could have been done legally, without victims, without military action, without losing Crimea. We would have never considered to even lift a finger if it hadn’t been for the bloody developments on Maidan.\nBecause we agreed with the fact that after the collapse of the Soviet Union our borders should be along the borders of former Union’s republics. We agreed to that. But we never agreed to NATO’s expansion and moreover we never agreed that Ukraine would be in NATO. We did not agree to NATO bases there without any discussion with us. For decades we kept urging them: don’t do this, don’t do that.\nAnd what triggered the latest events? Firstly, the current Ukrainian leadership declared that it would not implement the Minsk agreements, which had been signed, as you know, after the events of 2014, in Minsk, where the plan of a peaceful settlement in Donbass was set forth. But no, the current Ukrainian leadership, foreign minister, all other officials and then President himself said that they don’t like anything about the Minsk agreements. In other words, they were not going to implement them. A year or a year and a half ago, former leaders of Germany and France said openly to the whole world that they indeed signed the Minsk agreements but they never intended to implement them. They simply led us by the nose.\n"


------

## Flavored variations

In this section we show how the spoken parts can be rephrased in the style of certain political celebrities.

Here are examples of using LLM to rephrase Tucker Carlson's questions into the style of Hillary Clinton:

In [33]:
for _ in range(2):
    q = random.choice(tcQuestions)
    print('=' * 100)
    print("Tucker Carlson:", q)
    print('-' * 100)
    q2 = llm_synthesize(["Rephrase this question in the style of Hillary Clinton:", q])
    print("Hillary Clinton:", q2)


Tucker Carlson: Would you be willing to say, “Congratulations, NATO, you won?” And just keep the situation where it is now?
----------------------------------------------------------------------------------------------------
Hillary Clinton: Are you prepared to express your utmost admiration and extend heartfelt congratulations to NATO, acknowledging their resounding victory, and maintain the current state of affairs without any further alterations?
Tucker Carlson: Mr. President, thank you.
On February 24, 2022, you addressed your country in your nationwide address when the conflict in Ukraine started and you said that you were acting because you had come to the conclusion that the United States through NATO might initiate a quote, “surprise attack on our country.” And to American ears that sounds paranoid. Tell us why you believe the United States might strike Russia out of the blue. How did you conclude that?
---------------------------------------------------------------------------

Here are examples of using LLM to rephrase Vladimir Putin's answers into the style of Donald Trump:

In [34]:
[value for key, value in parts.items() if 'Putin' in key]

[" It's not that the United States was going to launch a surprise strike on Russia, I didn't say so. Are we having a talk show or a serious conversation?\n",
 ' You were initially trained in history, as far as I know?\n',
 ' So if you don’t mind I will take only 30 seconds or one minute of your time to give you a brief historical background.\n',
 " Let’s look where our relationship with Ukraine started from. Where does Ukraine come from?\nThe Russian state started to exist as a centralized state in 862. This is considered to be the year of creation of the Russian state because this year the townspeople of Novgorod (a city in the North-West of the country) invited Rurik, a Varangian prince from Scandinavia, to reign. In 1862, Russia celebrated the 1000th anniversary of its statehood, and in Novgorod there is a memorial dedicated to the 1000th anniversary of the country.\nIn 882, Rurik's successor Prince Oleg, who was, actually, playing the role of regent for Rurik’s young son because Ru

In [35]:
for _ in range(2):
    q = random.choice([value for key, value in parts.items() if 'Putin' in key])
    print('=' * 100)
    print("Vladimir Putin:", q)
    print('-' * 100)
    q2 = llm_synthesize(["Rephrase this question in the style of Hillary Clinton:", q])
    print("Donald Trump:", q2)

Vladimir Putin:  Exactly.

----------------------------------------------------------------------------------------------------
Donald Trump: Could you please rephrase this question in the style of Hillary Clinton?
Vladimir Putin:  (Laughing.) You are asking increasingly more complicated questions. To answer them, you need to be an expert in big numbers, big data and AI.
Mankind is currently facing many threats. Due to genetic research, it is now possible to create a superhuman, a specialized human being – a genetically engineered athlete, scientist, military man.
There are reports that Elon Musk has already had a chip implanted in the human brain in the USA.

----------------------------------------------------------------------------------------------------
Donald Trump: (Laughing.) You are posing quite intricate inquiries. To adequately respond, one must possess expertise in vast numerical concepts, extensive data analysis, and the realm of artificial intelligence.

Presently, human

-------

## References

### Articles

[AA1] Anton Antonov,
["Workflows with LLM functions"](https://rakuforprediction.wordpress.com/2023/08/01/workflows-with-llm-functions/),
(2023),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

[AA2] Anton Antonov,
["Day 21 – Using DALL-E models in Raku"](https://raku-advent.blog/2023/12/21/day-22-using-dall-e-models-in-raku/),
(2023),
[Raku Advent Calendar blog for 2023](https://raku-advent.blog/2023).

[OAIb1] OpenAI team,
["New models and developer products announced at DevDay"](https://openai.com/blog/new-models-and-developer-products-announced-at-devday),
(2023),
[OpenAI/blog](https://openai.com/blog).

### Packages

[AAp1] Anton Antonov,
[WWW::OpenAI](https://github.com/antononcube/Raku-WWW-OpenAI) Raku package,
(2023),
[GitHub/antononcube](https://github.com/antononcube).


[AAp2] Anton Antonov,
[WWW::PaLM](https://github.com/antononcube/Raku-WWW-PaLM) Raku package,
(2023),
[GitHub/antononcube](https://github.com/antononcube).


[AAp3] Anton Antonov,
[WWW::MistralAI](https://github.com/antononcube/Raku-WWW-MistralAI) Raku package,
(2023),
[GitHub/antononcube](https://github.com/antononcube).


[AAp4] Anton Antonov,
[WWW::MermaidInk](https://github.com/antononcube/Raku-WWW-MermaidInk) Raku package,
(2023),
[GitHub/antononcube](https://github.com/antononcube).


[AAp5] Anton Antonov,
[LLM::Functions](https://github.com/antononcube/Raku-LLM-Functions) Raku package,
(2023),
[GitHub/antononcube](https://github.com/antononcube).


[AAp6] Anton Antonov,
[Jupyter::Chatbook](https://github.com/antononcube/Raku-Jupyter-Chatbook) Raku package,
(2023),
[GitHub/antononcube](https://github.com/antononcube).


[AAp7] Anton Antonov,
[Image::Markup::Utilities](https://github.com/antononcube/Raku-Image-Markup-Utilities) Raku package,
(2023),
[GitHub/antononcube](https://github.com/antononcube).


### Videos

[AAv1] Anton Antonov,
["Jupyter Chatbook LLM cells demo (Raku)"](https://www.youtube.com/watch?v=cICgnzYmQZg)
(2023),
[YouTube/@AAA4Prediction](https://www.youtube.com/@AAA4prediction).

[AAv2] Anton Antonov,
["Jupyter Chatbook multi cell LLM chats teaser (Raku)"](https://www.youtube.com/watch?v=wNpIGUAwZB8),
(2023),
[YouTube/@AAA4Prediction](https://www.youtube.com/@AAA4prediction).

[AAv3] Anton Antonov
["Integrating Large Language Models with Raku"](https://www.youtube.com/watch?v=-OxKqRrQvh0),
(2023),
[YouTube/@therakuconference6823](https://www.youtube.com/@therakuconference6823).